In [ ]:
import keras
from keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Load data and normalize it
root_dir = "/kaggle/input"
train_data = pd.read_csv(root_dir + "/Kannada-MNIST/train.csv")
train_labels = train_data['label'].to_numpy()
train_pics = train_data.iloc[:,1:].to_numpy().reshape(-1,28,28,1)/255

val_data = pd.read_csv(root_dir + "/Kannada-MNIST/Dig-MNIST.csv")
val_labels = val_data['label'].to_numpy()
val_pics = val_data.iloc[:,1:].to_numpy().reshape(-1,28,28,1)/255

In [ ]:
print(train_pics.shape, val_pics.shape)

plt.figure(1)
plt.hist(train_labels, bins = 10)
plt.title("Histogram of the training data")
plt.xlabel("Labels")
plt.show()

plt.figure(2)
plt.imshow(train_pics[0].reshape(28,28), cmap = "gray")
plt.title(f"Example input labeled as {train_labels[0]}")
plt.show()

In [ ]:
#Data augmentation while training
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1)

In [ ]:
#Construct the model
model = keras.Sequential([
    layers.Input(shape=(28,28,1)),
    layers.Conv2D(32, 5, padding='same', activation='relu', name='conv1'),
    layers.Conv2D(32, 5, padding='same', activation='relu', name='conv2'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.4),
    
    layers.Conv2D(32, 3, padding='same', activation='relu', name='conv3'),
    layers.Conv2D(32, 3, padding='same', activation='relu', name='conv4'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.4),
    
    layers.Conv2D(64, 3, padding='same', activation='relu', name='conv5'),
    layers.Conv2D(64, 3, padding='same', activation='relu', name='conv6'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.4),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(10, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(1e-3), 
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics = ['accuracy'])
batch_size = 64
epochs = 20

#Validate model performance
log = model.fit(datagen.flow(train_pics,train_labels, batch_size=batch_size), 
                steps_per_epoch=len(train_data)/batch_size,
                epochs=epochs,verbose=1, validation_data=(val_pics,val_labels))

In [ ]:
#Plot learning curve
plt.figure(3)
plt.plot(log.history['accuracy'],label = 'train_accuracy')
plt.plot(log.history['val_accuracy'],label = 'val_accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show();

In [ ]:
#Plot confusion matrix
y_valid = model.predict(val_pics)
y_valid = np.argmax(y_valid, axis=1)
conf_matrix = confusion_matrix(val_labels, y_valid)
ax = plt.subplot()
sns.heatmap(conf_matrix, annot=True, ax=ax, cmap='Blues', fmt='3d')
#labels, title
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')

In [ ]:
#Merge training and validation data for the final training
merged_pics = np.vstack((train_pics,val_pics))
merged_label =  np.hstack((train_labels,val_labels))

In [ ]:
#Train with all the available data
model.compile(optimizer=keras.optimizers.Adam(1e-3), 
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics = ['accuracy'])
log = model.fit(datagen.flow(merged_pics,merged_label, batch_size=batch_size), 
                steps_per_epoch=len(train_data)/batch_size,
                epochs=epochs,verbose=2)

In [ ]:
#Load test data
test_data = pd.read_csv(root_dir + "/Kannada-MNIST/test.csv")
print(test_data.shape)
test_id = np.array(test_data.index)
test_pics = test_data.iloc[:,1:].to_numpy().reshape(-1,28,28,1)/255

In [ ]:
#Get predictions for the test data
test_prediction = model.predict(test_pics)
test_prediction = np.argmax(test_prediction, axis = 1)
res = pd.DataFrame(np.concatenate([test_id[:, np.newaxis], test_prediction[:,np.newaxis]], axis=1),
                   columns=["id", "label"])

In [ ]:
res

In [ ]:
res.to_csv('./submission.csv', index = False)